<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/BX-gpt-app/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 설치

In [1]:
!pip install -q openai
!pip install -q google.colab
!pip install -q pymongo
!pip install -q pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.1 MB/s eta 0:00:00


# Inference method

In [2]:
from openai import OpenAI
from google.colab import userdata

def get_completion(prompt: str) -> str:
    """
    Args:
        prompt: 프롬프트

    Returns: 답변 텍스트 데이터

    """

    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    client.close()

    return response.choices[0].message.content

# Prompt

In [11]:
PROMPT_PREFIX = """
document를 참고해 question과 answer 쌍을 생성하세요.
question은 document의 내용에 대해 물어보는 내용입니다.
answer은 question에 대해 document를 기반으로 답변하는 내용입니다.
question과 answer 쌍을 10개 생성하세요.

요구사항은 다음과 같습니다:
1. 문제의 다양성을 위해 question은 비슷한 단어로 반복하지 않습니다.
2. answer은 최대한 간결한 내용으로 작성되어야 합니다.
3. question은 document에 대해 물어보는 내용이며, 100단어 정도로 구성해주세요.
4. answer은 document를 기반으로 자세한 내용이 포함되도록 제공되어야하지만 100단어 정도로 구성해주세요.
5. 출력 형식은 JSON포맷을 따라야 하며, Indentation은 없도록 출력하세요.

출력 형식은 아래 포맷을 참조하세요:
{"question": "컴퓨터의 구성 요소는 어떻게 되어 있나요??", "answer": "CPU, 주기억장치, 보조기억장치 등으로 이루어져 있습니다."}
{"question": "Java와 Python의 차이는 무엇인가요?", "answer": "컴파일 언어와 인터프리터 언어의 차이가 있는데요. 우선..."}
"""

PROMPT_CONTENT_FORMAT = """
###document:
{document}
"""

# mongoDB

In [4]:
!curl ipecho.net/plain

34.90.35.86

In [5]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["spider504"]
collection = db["240703_movie_collection"]

Connection to MongoDB successful


In [35]:
def fetch_limited_documents(collection, start, end):
    try:
        limit = end - start + 1
        projection = {"_id": 0, "text": 1}
        documents_cursor = collection.find({}, projection).skip(start).limit(limit)
        return list(documents_cursor)
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

documents = fetch_limited_documents(collection, 401, 500)

cnt = 0

for document in documents:
    print(document)
    cnt += 1
    if cnt == 10:
        break

{'text': '  거래내역 데이터 구조\n거래내역은 거래기본, 입출내역, 거래확장정보로 구성되어 있다\n거래내역 데이터 구성항목은 다음과 같다\n| 구분             | 설명                                                                                                      | 주요항목                                                              |\n| ---------------- | --------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------- |\n| 거래기본         | - 거래의 기본적인 정보 관리                                                                               | 계좌번호, 고객번호, 거래일, 거래상태, 상대거래정보                    |\n| 입출내역 (Entry) | -\xa0하나의 거래에서 발생하는 입금/출금 내역을 금액유형별로 관리<br />-\xa0계약의 잔액갱신 및 회계처리시 이용 | 정기예금 해지 시 금액유형, 입출구분, 거래금액, 통화코드, 현금대체구분 |\n| 거래확장정보     | -\xa0거래 기본정보 이외에 추가적으로 관리가 필요한 정보 관리                                                | 전표구분코드, 해지구분코드                                            |\n정기예금 해지 시, 입출내역 (Entry) 데이터의 예시는 다음과 같다\n* 가정: 해지 원금이 백만원, 이자

In [36]:
print(len(documents))

33


# API call

In [37]:
results = []

curr = 0

for document in documents:
    if curr % 10 == 0:
      print(f"Processed {curr} documents")
    curr += 1

    PROMPT_CONTENT = "\n".join([
      PROMPT_CONTENT_FORMAT.format(document = document)
    ])
    instruction = PROMPT_PREFIX + PROMPT_CONTENT
    result = get_completion(instruction)
    results.append(result)

Processed 0 documents
Processed 10 documents
Processed 20 documents
Processed 30 documents


In [38]:
print(results[0][:2000])
print(len(result))

```json
{"question": "거래내역 데이터 구조는 어떻게 구성되어 있나요?", "answer": "거래내역은 거래기본, 입출내역, 거래확장정보로 구성되어 있습니다."}
{"question": "거래기본 항목에는 어떤 정보가 포함되나요?", "answer": "거래기본 항목에는 계좌번호, 고객번호, 거래일, 거래상태, 상대거래정보가 포함됩니다."}
{"question": "입출내역 항목은 어떤 정보를 관리하나요?", "answer": "입출내역 항목은 금액유형별로 입금/출금 내역을 관리하며, 정기예금 해지 시 금액유형, 입출구분, 거래금액, 통화코드, 현금대체구분을 포함합니다."}
{"question": "거래확장정보에는 어떤 항목이 포함되나요?", "answer": "거래확장정보에는 전표구분코드와 해지구분코드가 포함됩니다."}
{"question": "정기예금 해지 시 입출내역 데이터의 예시는 어떻게 되나요?", "answer": "정기예금 해지 시 입출내역 데이터의 예시는 원금 출금 1,000,000 KRW, 수신이자 출금 10,000 KRW, 이자소득세 입금 1,400 KRW입니다."}
{"question": "거래내역을 저장하고 조회하기 위해 필요한 개념은 무엇인가요?", "answer": "거래내역을 저장하고 조회하기 위해서는 거래기본, 입출내역, 거래확장정보의 개념을 숙지해야 합니다."}
{"question": "거래기본 정보는 어떤 목적으로 사용되나요?", "answer": "거래기본 정보는 거래의 기본적인 정보를 관리하는 목적으로 사용됩니다."}
{"question": "입출내역 데이터는 어떤 상황에서 사용되나요?", "answer": "입출내역 데이터는 하나의 거래에서 발생하는 입금/출금 내역을 금액유형별로 관리하고, 계약의 잔액갱신 및 회계처리 시 사용됩니다."}
{"question": "거래확장정보는 왜 필요한가요?", "answer": "거래확장정보는 거래 기본정보 이외에 추가적으로 관리가 필요한 정보를 관리하기 위해 필요합니다

# jsonl 파일 저장

In [39]:
data_path = "/content/drive/MyDrive/AI-modeling/BX-gpt-app/CBP-Certi-instruction-data-401toEND.jsonl"

with open(data_path, "w", encoding="utf-8") as f:
    for result in results:
        for line in result.split("\n"):
            f.write(line)
            f.write("\n")

# 전처리

`"{\"question\": \"어쩌구?\", \"answer\": \"저쩌구.\"}"`

위 포맷을 지켜야 함

In [40]:
data_path = "/content/drive/MyDrive/AI-modeling/BX-gpt-app/CBP-Certi-instruction-data-all.jsonl"

with open(data_path, "r", encoding="utf-8") as f:
  parsed = f.readlines()

제거 항목
- [
- ]
- ```
- ```json

In [53]:
# parsed = [lines.replace("\n", "") for lines in parsed if lines != "\n"]

preprocessed_parsed = []

for line in parsed:
  if line.startswith("[") or line.endswith("]") or line.startswith("```") or line.endswith("```json"):
    continue

  if line.startswith("]"):
    continue

  if line == "\n":
    continue

  line = line.replace("},", "}")
  line = line.replace("\n", "")

  preprocessed_parsed.append(line)
  # print(line)

In [55]:
preprocessed_data_path = "/content/drive/MyDrive/AI-modeling/BX-gpt-app/CBP-Certi-instruction-data-preprocessed.jsonl"

with open(preprocessed_data_path, "w", encoding="utf-8") as f:
  for line in preprocessed_parsed:
      f.write(line)
      f.write("\n")

# DataFrame

`train.ipynb 에서 활용`

In [56]:
import pandas as pd
import json

# 데이터 읽기
data = []
with open(preprocessed_data_path, "r", encoding="utf-8") as f:
    for line in f:
        json_line = json.loads(line)
        data.append({
            "question": json_line["question"],
            "answer": json_line["answer"]
        })

# 데이터셋 생성
df = pd.DataFrame(data)
# dataset = Dataset.from_pandas(df)
df.head()

,question,answer
0,우리나라 은행 IT 시스템의 발전 과정은 어떻게 되나요?,"1970년대 과목 별 온라인 시스템, 1980년대 종합 온라인 시스템, 1990년대..."
1,1세대 은행 IT 시스템의 주요 특징은 무엇인가요?,"전국 온라인 처리와 대량 거래 처리가 가능했으며, Mainframe과 Assembl..."
2,2세대 은행 IT 시스템의 비즈니스 모델 변화는 무엇이었나요?,"창구 역할 변화, One stop service, 온라인 고객 정보 활용, 경영 정..."
3,3세대 은행 IT 시스템의 주요 기술 변화는 무엇이었나요?,"C/S(hetero 환경), HLL, C, web, Package 솔루션, CBD ..."
4,포스트 차세대 은행 IT 시스템의 비즈니스 모델 변화는 무엇인가요?,"융복합상품서비스, 지주사 차원 고객 통합서비스, 앱 금융서비스, 국외 업무 확산, ..."


In [57]:
df.shape

(5100, 2)